In [1]:
#Importing libraries needed to complete web scraping
from bs4 import BeautifulSoup as soup
import requests
import re

In [2]:
#url for desired business. Yelp updated the tags so they need to be updated if it isn't finding the text.
url = 'https://www.yelp.com/biz/vault-brewing-yardley'
response = requests.get(url)
s = soup(response.text,'html.parser')

In [3]:
response.status_code #used to confirm whether data can be scraped. 200 indicats that scraping will be accepted.

200

In [4]:
num_reviews = int(s.find('p',attrs = {'class':"lemon--p__373c0__3Qnnj text__373c0__2Kxyz text-color--mid__373c0__jCeOG text-align--left__373c0__2XGa- text-size--large__373c0__3t60B"}).text.strip(' reviews'))

In [20]:
print(str(num_reviews)+' reviews posted about this restaurant')
url_list = []
#creating list of all the urls to call in order to scrape reviews on different pages
for i in range(0, num_reviews,20):
    url_list.append(url+'?start='+str(i))

568 reviews posted about this restaurant


In [6]:
#part of webpage that highlights the entire review
reviews = s.find_all('div',attrs = {'class':"lemon--div__373c0__1mboc review__373c0__13kpL sidebarActionsHoverTarget__373c0__2kfhE arrange__373c0__2C9bH gutter-2__373c0__1DiLQ grid__373c0__1Pz7f layout-stack-small__373c0__27wVp border-color--default__373c0__3-ifU"})

In [8]:
review = reviews[2]
#To protect privacy, Yelp user names have not been included in the file. However, source code is below.
#username = review.find('span',attrs = {'class':"lemon--span__373c0__3997G text__373c0__2pB8f fs-block text-color--inherit__373c0__w_15m text-align--left__373c0__2pnx_ text-weight--bold__373c0__3HYJa"}).string
#print(username)

In [9]:
location = review.find('span',attrs = {'class':"lemon--span__373c0__3997G text__373c0__2Kxyz text-color--normal__373c0__3xep9 text-align--left__373c0__2XGa- text-weight--bold__373c0__1elNz text-size--small__373c0__3NVWO"}).get_text().strip()

In [10]:
print(location)

Philadelphia, PA


In [11]:
#getting the rating and doing string manipulation to change the data type to float
rating = float(review.select('[aria-label*=rating]')[0]['aria-label'].strip(' star rating'))

In [12]:
print(rating)

3.0


In [13]:
#Extracting the date the review was posted on Yelp
date = review.find('span',attrs = {'class':"lemon--span__373c0__3997G text__373c0__2Kxyz text-color--mid__373c0__jCeOG text-align--left__373c0__2XGa-"}).get_text()
print(date)

6/21/2020


In [14]:
#Extracting the text of the review
content = review.find('p',attrs = {'class':"lemon--p__373c0__3Qnnj text__373c0__2Kxyz comment__373c0__3EKjH text-color--normal__373c0__3xep9 text-align--left__373c0__2XGa-"}).text

In [15]:
content

"Food is good, service subpar. We arrived and were told to wear a mask inside when going to the host stand. They weren't enforcing this, and most people weren't even wearing a mask. Whatever, but if you are going to have a rule, please enforce it. We were sat at our table, given one paper menu to share amongst the whole table of 6. Who knows if it was a new menu. It felt damp. Doesn't that defeat the purpose of a reusable menu? So everyone can have their own and toss it out after?Service was so slow, no water refills, waitress would leave before we could even ask for extra napkins and water.The duck and apple pizza was excellent. I did enjoy the pork belly fingerling fries as well. I enjoyed my blackberry drip beer, although I tasted none of the flavors noted."

In [17]:
import csv
import time
import random

def scrape_single_page(reviews, csvwriter):
    for review in reviews:
        dic = {}
        #Code copied from previous steps
        #username = review.find('span',attrs = {'class':"lemon--span__373c0__3997G text__373c0__2pB8f fs-block text-color--inherit__373c0__w_15m text-align--left__373c0__2pnx_ text-weight--bold__373c0__3HYJa"}).string
        try:
            location = review.find('span',attrs = {'class':"lemon--span__373c0__3997G text__373c0__2Kxyz text-color--normal__373c0__3xep9 text-align--left__373c0__2XGa- text-weight--bold__373c0__1elNz text-size--small__373c0__3NVWO"}).get_text().strip()  
        except:
            location = ''
        date = review.find('span',attrs = {'class':"lemon--span__373c0__3997G text__373c0__2Kxyz text-color--mid__373c0__jCeOG text-align--left__373c0__2XGa-"}).get_text()
        rating = float(review.select('[aria-label*=rating]')[0]['aria-label'].strip(' star rating'))
        content = review.find('p',attrs = {'class':"lemon--p__373c0__3Qnnj text__373c0__2Kxyz comment__373c0__3EKjH text-color--normal__373c0__3xep9 text-align--left__373c0__2XGa-"}).text
        #Assign Values to the dictionary
        #dic['username']=username Commented out to protect yelp user privacy
        dic['location']=location
        dic['date'] = date
        dic['rating'] = rating
        dic['content']=content
        review_writer.writerow(dic.values())
header = ['Location','Date','Rating','Content']
with open('vaultbrewing.csv','w',encoding = 'utf-8') as csvfile:
    review_writer = csv.writer(csvfile)
    review_writer.writerow(header)
    for index, url in enumerate(url_list):
        response = requests.get(url).text
        s = soup(response, 'html.parser')
        reviews = s.find_all('div',attrs = {'class':"lemon--div__373c0__1mboc review__373c0__13kpL sidebarActionsHoverTarget__373c0__2kfhE arrange__373c0__2C9bH gutter-2__373c0__1DiLQ grid__373c0__1Pz7f layout-stack-small__373c0__27wVp border-color--default__373c0__3-ifU"})        
        scrape_single_page(reviews, review_writer)
        #Random sleep to avoid getting banned from the server
        time.sleep(random.randint(1,5))
        #log the progress
        print('finished page '+str(index+1) + ' of '+ str(len(url_list)))

finished page 1 of 29
finished page 2 of 29
finished page 3 of 29
finished page 4 of 29
finished page 5 of 29
finished page 6 of 29
finished page 7 of 29
finished page 8 of 29
finished page 9 of 29
finished page 10 of 29
finished page 11 of 29
finished page 12 of 29
finished page 13 of 29
finished page 14 of 29
finished page 15 of 29
finished page 16 of 29
finished page 17 of 29
finished page 18 of 29
finished page 19 of 29
finished page 20 of 29
finished page 21 of 29
finished page 22 of 29
finished page 23 of 29
finished page 24 of 29
finished page 25 of 29
finished page 26 of 29
finished page 27 of 29
finished page 28 of 29
finished page 29 of 29
